In [204]:
import numpy as np
import pandas as pd
import geopandas as gpd

import folium
from folium.plugins import DualMap
import branca.colormap as cm

import re

# ***Note:*** cd to your own directory

In [205]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Personal projects/NYC Election/primary-vs-general-mayoral-enr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Personal projects/NYC Election/primary-vs-general-mayoral-enr


# **Map the Result**
## Mamdani vs. Cuomo

## Prepare

### Load Data for Mapping

In [206]:
# Read the primary election results CSV into a DataFrame
primary_csv_path = "../primary-mayoral-enr-web-archive/output_files/ed_map_gdf_results_2025-06-25_00_33_06_EST.csv"
ed_map_gdf_primary = pd.read_csv(primary_csv_path)

# Read the general election results CSV into a DataFrame
general_csv_path = "../general-mayoral-enr/output_files/ed_map_gdf_results_2025-11-05_00_31_51_EST.csv"
ed_map_gdf_general = pd.read_csv(general_csv_path)

print(f"Loaded primary election data with {len(ed_map_gdf_primary)} rows and {len(ed_map_gdf_primary.columns)} columns.")
print(f"Loaded general election data with {len(ed_map_gdf_general)} rows and {len(ed_map_gdf_general.columns)} columns.")

# NOTE: If these CSVs contain a 'geometry' column in WKT or GeoJSON format,
# they will need to be converted to actual geometry objects to become GeoDataFrames.

# Check if a 'geometry' column exists and is not null in primary dataframe
if 'geometry' in ed_map_gdf_primary.columns and ed_map_gdf_primary['geometry'].notna().any():
    try:
        ed_map_gdf_primary['geometry'] = gpd.GeoSeries.from_wkt(ed_map_gdf_primary['geometry'])
        ed_map_gdf_primary = gpd.GeoDataFrame(ed_map_gdf_primary, geometry='geometry', crs="EPSG:4326") # WGS84
        print("Primary election data converted to GeoDataFrame from WKT geometry.")
    except Exception as e:
        print(f"Could not convert primary geometry from WKT: {e}. Keeping as DataFrame.")
else:
    print("No 'geometry' column found or it's empty in primary data. Keeping as DataFrame.")

# Check if a 'geometry' column exists and is not null in general dataframe
if 'geometry' in ed_map_gdf_general.columns and ed_map_gdf_general['geometry'].notna().any():
    try:
        ed_map_gdf_general['geometry'] = gpd.GeoSeries.from_wkt(ed_map_gdf_general['geometry'])
        ed_map_gdf_general = gpd.GeoDataFrame(ed_map_gdf_general, geometry='geometry', crs="EPSG:4326") # WGS84
        print("General election data converted to GeoDataFrame from WKT geometry.")
    except Exception as e:
        print(f"Could not convert general geometry from WKT: {e}. Keeping as DataFrame.")
else:
    print("No 'geometry' column found or it's empty in general data. Keeping as DataFrame.")

Loaded primary election data with 4264 rows and 44 columns.
Loaded general election data with 4264 rows and 32 columns.
Primary election data converted to GeoDataFrame from WKT geometry.
General election data converted to GeoDataFrame from WKT geometry.


In [207]:
display(ed_map_gdf_primary.head(2))
display(ed_map_gdf_general.head(2))

,ElectDist,geometry,Reported,Total Votes,Zohran Kwame Mamdani,Andrew M. Cuomo,Brad Lander,Adrienne E. Adams,Scott M. Stringer,Zellnor Myrie,...,Brad Lander (%)_fmt,Adrienne E. Adams (%)_fmt,Scott M. Stringer (%)_fmt,Zellnor Myrie (%)_fmt,Whitney R. Tilson (%)_fmt,Michael Blake (%)_fmt,Jessica Ramos (%)_fmt,Paperboy Love Prince (%)_fmt,WRITE-IN (%)_fmt,Selma K. Bartholomew (%)_fmt
0,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.561...",99.00%,198,31,121,23,0,7,1,...,11.62%,0.00%,3.54%,0.51%,2.02%,1.52%,0.00%,3.54%,0.51%,0.00%
1,23002,"POLYGON ((-73.91017 40.56492, -73.91039 40.564...",99.00%,168,25,108,14,6,6,0,...,8.33%,3.57%,3.57%,0.00%,2.98%,0.60%,0.00%,0.60%,0.60%,0.60%


,ElectDist,geometry,Reported,Total Votes,Zohran Kwame Mamdani,Andrew M. Cuomo,Curtis A. Sliwa,Eric L. Adams,WRITE-IN,Irene Estrada,...,winning_candidate,winning_percentage_fmt,Zohran Kwame Mamdani (%)_fmt,Andrew M. Cuomo (%)_fmt,Curtis A. Sliwa (%)_fmt,Eric L. Adams (%)_fmt,WRITE-IN (%)_fmt,Irene Estrada (%)_fmt,Jim Walden (%)_fmt,Joseph Hernandez (%)_fmt
0,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.561...",99.00%,1094,78,662,346,1,0,5,...,Andrew M. Cuomo,60.51%,7.13%,60.51%,31.63%,0.09%,0.00%,0.46%,0.18%,0.00%
1,23002,"POLYGON ((-73.91017 40.56492, -73.91039 40.564...",99.00%,1078,75,627,369,1,0,4,...,Andrew M. Cuomo,58.16%,6.96%,58.16%,34.23%,0.09%,0.00%,0.37%,0.09%,0.09%


### Define Colormaps

Import the 'branca.colormap' module and define two colormaps: 'YlOrRd_05' for Zohran Kwame Mamdani and 'YlGnBu_05' for Andrew M. Cuomo, scaled from 0 to 100 for percentage representation, and add captions for the legends.


In [208]:
# Define colormap for Zohran Kwame Mamdani
mamdani_colormap = cm.linear.YlOrRd_05.scale(vmin=0, vmax=100)
mamdani_colormap.caption = 'Zohran Kwame Mamdani (%)'
print(f"Defined mamdani_colormap with caption: {mamdani_colormap.caption}")

# Define colormap for Andrew M. Cuomo
cuomo_colormap = cm.linear.YlGnBu_05.scale(vmin=0, vmax=100)
cuomo_colormap.caption = 'Andrew M. Cuomo (%)'
print(f"Defined cuomo_colormap with caption: {cuomo_colormap.caption}")

Defined mamdani_colormap with caption: Zohran Kwame Mamdani (%)
Defined cuomo_colormap with caption: Andrew M. Cuomo (%)


### Create Custom Style Function

A Python function that determines the fill color, fill opacity, line color, line weight, and dash array for each GeoJSON feature based on the winning candidate. It uses the defined colormaps for Mamdani and Cuomo, and a red dashed line for all other winners or EDs with zero total votes.


This function takes a GeoJSON feature as input and, based on the `winning_candidate` and `winning_percentage` in its properties, it returns a dictionary of styling attributes (fill color, opacity, line color, weight, and dash array).


In [209]:
def style_function(feature):
    winning_candidate = feature['properties']['winning_candidate']
    winning_percentage = feature['properties']['winning_percentage']

    if winning_candidate == 'Zohran Kwame Mamdani':
        fill_color = mamdani_colormap(winning_percentage)
        fill_opacity = 0.7
        line_color = 'black'
        line_weight = 0.5
        dash_array = ''
    elif winning_candidate == 'Andrew M. Cuomo':
        fill_color = cuomo_colormap(winning_percentage)
        fill_opacity = 0.7
        line_color = 'black'
        line_weight = 0.5
        dash_array = ''
    else:
        fill_color = 'transparent'
        fill_opacity = 0.4
        line_color = 'red'
        line_weight = 0.5
        dash_array = '5, 5'

    return {
        'fillColor': fill_color,
        'fillOpacity': fill_opacity,
        'color': line_color,
        'weight': line_weight,
        'dashArray': dash_array
    }
print("Defined custom style_function for mapping.")

Defined custom style_function for mapping.


## Generate Folium Map

Create a Folium map centered on the NYC area. Use 'folium.features.GeoJson' with the custom style function to render the Election Districts with the conditional coloring. Configure the tooltips to display relevant election results, including the winning candidate and percentage. Add the colormap legends to the map.


### Map 1: Primary

In [210]:
# Ensure the GeoDataFrame is in a geographic coordinate system (e.g., WGS84 for Folium)
# If it's not, reproject it:
if ed_map_gdf_primary.crs and ed_map_gdf_primary.crs.to_epsg() != 4326:
    ed_map_gdf_primary = ed_map_gdf_primary.to_crs(epsg=4326)

# Calculate the centroid to center the map using union_all() as recommended
center_point = ed_map_gdf_primary.geometry.union_all().centroid

# Create a Folium map centered on the area
m1 = folium.Map(location=[center_point.y, center_point.x], tiles="Cartodb Positron", zoom_start=11)

# Create formatted percentage columns for tooltips
ed_map_gdf_primary['winning_percentage_fmt'] = ed_map_gdf_primary['winning_percentage'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

candidate_pct_columns = [col for col in ed_map_gdf_primary.columns if col.endswith(' (%)')]
for col in candidate_pct_columns:
    ed_map_gdf_primary[f'{col}_fmt'] = ed_map_gdf_primary[col].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

# Define columns for the tooltip, using formatted ones where applicable
tooltip_cols = [
    'ElectDist', 'Total Votes', 'winning_candidate', 'winning_percentage_fmt',
    'Zohran Kwame Mamdani (%)_fmt', 'Andrew M. Cuomo (%)_fmt', 'Brad Lander (%)_fmt', 'Adrienne E. Adams (%)_fmt',
    'Scott M. Stringer (%)_fmt', 'Zellnor Myrie (%)_fmt', 'Whitney R. Tilson (%)_fmt', 'Michael Blake (%)_fmt', 'Jessica Ramos (%)_fmt',
    'Paperboy Love Prince (%)_fmt', 'WRITE-IN (%)_fmt', 'Selma K. Bartholomew (%)_fmt'
]

# Define aliases for the tooltip fields
tooltip_aliases = [
    'Election District', 'Total Votes', 'Winning Candidate', 'Winning Percentage',
    'Zohran Kwame Mamdani', 'Andrew M. Cuomo', 'Brad Lander', 'Adrienne E. Adams',
    'Scott M. Stringer', 'Zellnor Myrie', 'Whitney R. Tilson', 'Michael Blake', 'Jessica Ramos',
    'Paperboy Love Prince', 'WRITE-IN', 'Selma K. Bartholomew'
]

# Add GeoJson layer with custom styling and tooltips
folium.GeoJson(
    ed_map_gdf_primary.__geo_interface__,
    name='Election Districts',
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols,
        aliases=tooltip_aliases, # Add the aliases here
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    )
).add_to(m1)

# Add colormap legends to the map
mamdani_colormap.add_to(m1)
cuomo_colormap.add_to(m1)

# Add a layer control to toggle layers if desired
folium.LayerControl().add_to(m1)

# Display the map
m1

Output hidden; open in https://colab.research.google.com to view.

### Map 2

In [211]:
# Ensure the GeoDataFrame is in a geographic coordinate system (e.g., WGS84 for Folium)
# If it's not, reproject it:
if ed_map_gdf_general.crs and ed_map_gdf_general.crs.to_epsg() != 4326:
    ed_map_gdf_general = ed_map_gdf_general.to_crs(epsg=4326)

# Calculate the ed_map_gdf_general to center the map using union_all() as recommended
center_point = ed_map_gdf_general.geometry.union_all().centroid

# Create a Folium map centered on the area
m2 = folium.Map(location=[center_point.y, center_point.x], tiles="Cartodb Positron", zoom_start=11)

# Create formatted percentage columns for tooltips
ed_map_gdf_general['winning_percentage_fmt'] = ed_map_gdf_general['winning_percentage'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

candidate_pct_columns = [col for col in ed_map_gdf_general.columns if col.endswith(' (%)')]
for col in candidate_pct_columns:
    ed_map_gdf_general[f'{col}_fmt'] = ed_map_gdf_general[col].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

# Define columns for the tooltip, using formatted ones where applicable
tooltip_cols = [
    'ElectDist', 'Total Votes', 'winning_candidate', 'winning_percentage_fmt',
    'Zohran Kwame Mamdani (%)_fmt', 'Andrew M. Cuomo (%)_fmt', 'Curtis A. Sliwa (%)_fmt',
    'Eric L. Adams (%)_fmt', 'WRITE-IN (%)_fmt', 'Irene Estrada (%)_fmt', 'Jim Walden (%)_fmt', 'Joseph Hernandez (%)_fmt'
]

# Define aliases for the tooltip fields
tooltip_aliases = [
    'Election District', 'Total Votes', 'Winning Candidate', 'Winning Percentage',
    'Zohran Kwame Mamdani', 'Andrew M. Cuomo', 'Curtis A. Sliwa',
    'Eric L. Adams', 'WRITE-IN', 'Irene Estrada', 'Jim Walden', 'Joseph Hernandez'
]

# Add GeoJson layer with custom styling and tooltips
folium.GeoJson(
    ed_map_gdf_general.__geo_interface__,
    name='Election Districts',
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols,
        aliases=tooltip_aliases, # Add the aliases here
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    )
).add_to(m2)

# Add colormap legends to the map
mamdani_colormap.add_to(m2)
cuomo_colormap.add_to(m2)

# Add a layer control to toggle layers if desired
folium.LayerControl().add_to(m2)

# Display the map
m2

Output hidden; open in https://colab.research.google.com to view.

### Map 1 and Map 2

Combine two existing Folium maps (`m1` and `m2`) into a `folium.plugins.DualMap`:

1.  **Initialize `folium.plugins.DualMap`** using a `location` (e.g., the center point of the maps) and a `layout` (e.g., 'horizontal').
2.  **Transfer the GeoJson layers** (along with their tooltips) from the original `m1` map to the `dual_map.m1` object.
3.  **Transfer the GeoJson layers** (along with their tooltips) from the original `m2` map to the `dual_map.m2` object.
4.  **Add the `mamdani_colormap` and `cuomo_colormap` legends** to the `dual_map` (specifically, to one of its internal maps, like `dual_map.m1`, so they are displayed).

In [212]:
# Ensure the GeoDataFrames are in a geographic coordinate system (e.g., WGS84 for Folium)
# If not, reproject them (already handled in previous cells, but good to ensure if running standalone)
if ed_map_gdf_primary.crs and ed_map_gdf_primary.crs.to_epsg() != 4326:
    ed_map_gdf_primary = ed_map_gdf_primary.to_crs(epsg=4326)

if ed_map_gdf_general.crs and ed_map_gdf_general.crs.to_epsg() != 4326:
    ed_map_gdf_general = ed_map_gdf_general.to_crs(epsg=4326)

# Calculate a common centroid to center the dual map
# Using the primary map's centroid as it was used for m1 initially
center_point = ed_map_gdf_primary.geometry.union_all().centroid

# Create a DualMap object with the specified location and layout
dual_map = DualMap(location=[center_point.y-0.02, center_point.x-0.05], layout='horizontal', tiles="Cartodb Positron", zoom_start=11)

# Define the highlight function for hover effect
highlight_function = lambda x: {
    'fillColor': '#ab82c5',
    'color': '#7b5aa6',
    'fillOpacity': 0.6,
    'weight': 2,
    'dashArray': ''
}

# --- Populate dual_map.m1 (Primary Election Map) ---

# Create formatted percentage columns for tooltips for primary data if not already existing
# (These were created in the original m1 cell, but ensuring their presence for this block)
if 'winning_percentage_fmt' not in ed_map_gdf_primary.columns:
    ed_map_gdf_primary['winning_percentage_fmt'] = ed_map_gdf_primary['winning_percentage'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')
primary_candidate_pct_cols = [col for col in ed_map_gdf_primary.columns if col.endswith(' (%)')]
for col in primary_candidate_pct_cols:
    if f'{col}_fmt' not in ed_map_gdf_primary.columns:
        ed_map_gdf_primary[f'{col}_fmt'] = ed_map_gdf_primary[col].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

# Define columns and aliases for the primary map tooltip
tooltip_cols_primary = [
    'ElectDist', 'Total Votes', 'winning_candidate', 'winning_percentage_fmt',
    'Zohran Kwame Mamdani (%)_fmt', 'Andrew M. Cuomo (%)_fmt', 'Brad Lander (%)_fmt', 'Adrienne E. Adams (%)_fmt',
    'Scott M. Stringer (%)_fmt', 'Zellnor Myrie (%)_fmt', 'Whitney R. Tilson (%)_fmt', 'Michael Blake (%)_fmt', 'Jessica Ramos (%)_fmt',
    'Paperboy Love Prince (%)_fmt', 'WRITE-IN (%)_fmt', 'Selma K. Bartholomew (%)_fmt'
]

tooltip_aliases_primary = [
    'Election District', 'Total Votes', 'Winning Candidate', 'Winning Percentage',
    'Zohran Kwame Mamdani', 'Andrew M. Cuomo', 'Brad Lander', 'Adrienne E. Adams',
    'Scott M. Stringer', 'Zellnor Myrie', 'Whitney R. Tilson', 'Michael Blake', 'Jessica Ramos',
    'Paperboy Love Prince', 'WRITE-IN', 'Selma K. Bartholomew'
]

# Add GeoJson layer for primary data to dual_map.m1
folium.GeoJson(
    ed_map_gdf_primary.__geo_interface__,
    name='Primary Election Districts',
    style_function=style_function,
    highlight_function=highlight_function, # Add highlight function here
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols_primary,
        aliases=tooltip_aliases_primary,
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    )
).add_to(dual_map.m1)

# --- Populate dual_map.m2 (General Election Map) ---

# Create formatted percentage columns for tooltips for general data if not already existing
# (These were created in the original m2 cell, but ensuring their presence for this block)
if 'winning_percentage_fmt' not in ed_map_gdf_general.columns:
    ed_map_gdf_general['winning_percentage_fmt'] = ed_map_gdf_general['winning_percentage'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')
general_candidate_pct_cols = [col for col in ed_map_gdf_general.columns if col.endswith(' (%)')]
for col in general_candidate_pct_cols:
    if f'{col}_fmt' not in ed_map_gdf_general.columns:
        ed_map_gdf_general[f'{col}_fmt'] = ed_map_gdf_general[col].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

# Define columns and aliases for the general map tooltip
tooltip_cols_general = [
    'ElectDist', 'Total Votes', 'winning_candidate', 'winning_percentage_fmt',
    'Zohran Kwame Mamdani (%)_fmt', 'Andrew M. Cuomo (%)_fmt', 'Curtis A. Sliwa (%)_fmt',
    'Eric L. Adams (%)_fmt', 'WRITE-IN (%)_fmt', 'Irene Estrada (%)_fmt', 'Jim Walden (%)_fmt', 'Joseph Hernandez (%)_fmt'
]

tooltip_aliases_general = [
    'Election District', 'Total Votes', 'Winning Candidate', 'Winning Percentage',
    'Zohran Kwame Mamdani', 'Andrew M. Cuomo', 'Curtis A. Sliwa',
    'Eric L. Adams', 'WRITE-IN', 'Irene Estrada', 'Jim Walden', 'Joseph Hernandez'
]

# Add GeoJson layer for general data to dual_map.m2
folium.GeoJson(
    ed_map_gdf_general.__geo_interface__,
    name='General Election Districts',
    style_function=style_function,
    highlight_function=highlight_function, # Add highlight function here
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols_general,
        aliases=tooltip_aliases_general,
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    )
).add_to(dual_map.m2)

# Generate HTML for each colormap
mamdani_legend_html = mamdani_colormap._repr_html_()
cuomo_legend_html = cuomo_colormap._repr_html_()

# Combine them in a custom HTML string with CSS for positioning
custom_legend_html = f"""
<div style="position: fixed;
            bottom: 20px; right: 5px; width: 466px; height: 230px;
            border:0; z-index:9999; font-size:12px;
            background-color:#fafaf8; opacity:0.8;">
  <div style="padding: 10px;">
    <div style="text-align: center;">
      <h2>Map the Election: Mamdani vs. Cuomo</h2>
      <h3>⇦&nbsp;&nbsp;Primary vs. General&nbsp;&nbsp;⇨</h3>
      {mamdani_legend_html}
      <br>
      {cuomo_legend_html}
      <br>
    </div>
    <br>
    <div style="line-height: 1.25;">
      <p><b>Data Source:</b> NYC Board of Elections Unofficial Election Night Results (uncertified and unranked); NYC Department of City Planning</p>
      <p>&copy;&nbsp;<a href="https://r-li.com">Ruoyu Li</a>&nbsp;&nbsp;|&nbsp;&nbsp;Project Repository: <a href="https://github.com/RY-Li/map-nyc-election/">RY-Li/map-nyc-election</a></p>
    </div>
  </div>
</div>
"""

# Create a folium.Html object and add it to the map's figure
dual_map.m2.get_root().html.add_child(folium.Element(custom_legend_html))


# Display the dual map
dual_map

Output hidden; open in https://colab.research.google.com to view.

## Export as HTML


In [213]:
dual_map.save('dual_election_map.html')
print("Dual map saved as 'dual_election_map.html'")

Dual map saved as 'dual_election_map.html'


# **Map the Vote Change**

## Filter and Recalculate Data for Two-Way Vote Share Change

In [214]:
# ---- Filter Data ----
cols_to_keep = ['ElectDist', 'geometry', 'Zohran Kwame Mamdani', 'Andrew M. Cuomo']
ed_map_gdf_primary_2way = ed_map_gdf_primary[cols_to_keep].copy()
ed_map_gdf_general_2way = ed_map_gdf_general[cols_to_keep].copy()

# ---- Compute Total Votes per ED ----
cand_cols = [c for c in ed_map_gdf_primary_2way.columns if c not in ("ElectDist", "geometry")]
ed_map_gdf_primary_2way["Total Votes"] = ed_map_gdf_primary_2way[cand_cols].sum(axis=1)
ed_map_gdf_general_2way["Total Votes"] = ed_map_gdf_general_2way[cand_cols].sum(axis=1)

# ---- Compute Percentage ----
# Percent columns (rounded to 2 decimals, guard against divide-by-zero)
ed_map_gdf_primary_2way["Mamdani(%)_Primary"] = np.where(
    ed_map_gdf_primary_2way["Total Votes"].eq(0), 0.0, (ed_map_gdf_primary_2way['Zohran Kwame Mamdani'] / ed_map_gdf_primary_2way["Total Votes"]) * 100.0
).round(2)
ed_map_gdf_general_2way["Mamdani(%)_General"] = np.where(
    ed_map_gdf_general_2way["Total Votes"].eq(0), 0.0, (ed_map_gdf_general_2way['Zohran Kwame Mamdani'] / ed_map_gdf_general_2way["Total Votes"]) * 100.0
).round(2)

display(ed_map_gdf_primary_2way.head(2))
display(ed_map_gdf_general_2way.head(2))

,ElectDist,geometry,Zohran Kwame Mamdani,Andrew M. Cuomo,Total Votes,Mamdani(%)_Primary
0,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.561...",31,121,152,20.39
1,23002,"POLYGON ((-73.91017 40.56492, -73.91039 40.564...",25,108,133,18.80


,ElectDist,geometry,Zohran Kwame Mamdani,Andrew M. Cuomo,Total Votes,Mamdani(%)_General
0,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.561...",78,662,740,10.54
1,23002,"POLYGON ((-73.91017 40.56492, -73.91039 40.564...",75,627,702,10.68


In [215]:
# ---- Merge ----
cols_to_keep_1 = ['ElectDist', 'geometry', 'Mamdani(%)_Primary']
cols_to_keep_2 = ['ElectDist', 'Mamdani(%)_General']
ed_map_gdf_primary_2way_merge = ed_map_gdf_primary_2way[cols_to_keep_1]
ed_map_gdf_general_2way_merge = ed_map_gdf_general_2way[cols_to_keep_2]
ed_map_gdf_2way = ed_map_gdf_primary_2way_merge.merge(ed_map_gdf_general_2way_merge, on='ElectDist', how='outer')
display(ed_map_gdf_2way.head(2))

,ElectDist,geometry,Mamdani(%)_Primary,Mamdani(%)_General
0,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.561...",20.39,10.54
1,23002,"POLYGON ((-73.91017 40.56492, -73.91039 40.564...",18.80,10.68


In [216]:
# ---- Calculate change ----
# Calculate the raw difference first
raw_change = ed_map_gdf_2way['Mamdani(%)_General'] - ed_map_gdf_2way['Mamdani(%)_Primary']

# Set Change to NaN if either General or Primary share is 0
condition = (ed_map_gdf_2way['Mamdani(%)_General'] == 0) | (ed_map_gdf_2way['Mamdani(%)_Primary'] == 0)
ed_map_gdf_2way['Mamdani(%)_Change'] = np.where(condition, np.nan, raw_change)

display(ed_map_gdf_2way.head(2))

,ElectDist,geometry,Mamdani(%)_Primary,Mamdani(%)_General,Mamdani(%)_Change
0,23001,"POLYGON ((-73.92033 40.56223, -73.92078 40.561...",20.39,10.54,-9.85
1,23002,"POLYGON ((-73.91017 40.56492, -73.91039 40.564...",18.80,10.68,-8.12


## Map the change

In [217]:
# ---- Calculate min/max for Mamdani(%)_Change (for reference) ----
min_change = ed_map_gdf_2way['Mamdani(%)_Change'].min()
max_change = ed_map_gdf_2way['Mamdani(%)_Change'].max()

print(f"Min Mamdani(%)_Change: {min_change:.2f}")
print(f"Max Mamdani(%)_Change: {max_change:.2f}")

Min Mamdani(%)_Change: -79.41
Max Mamdani(%)_Change: 53.33


In [220]:
# ---- Define Colormaps ----
limit = 40

# Positive Change: YlOrRd_05 scaled 0 to limit
# 0 (Yellow) -> limit (Red)
pos_colormap = cm.linear.YlOrRd_05.scale(vmin=0, vmax=limit)
pos_colormap.caption = '' # Remove default caption
pos_colormap.width = 180  # Adjust width

# Negative Change Map (Used for Filling Polygons)
# 0 (Yellow) -> limit (Blue)
neg_colormap = cm.linear.YlGnBu_05.scale(vmin=0, vmax=limit)

# Negative Change Legend (Visual Only)
neg_colors_reversed = list(cm.linear.YlGnBu_05.colors)[::-1]
neg_colormap_legend = cm.LinearColormap(colors=neg_colors_reversed, vmin=-limit, vmax=0)
neg_colormap_legend.caption = '' # Remove default caption
neg_colormap_legend.width = 180  # Adjust width

# ---- Define functions ----
def change_style_function(feature):
    mamdani_change = feature['properties']['Mamdani(%)_Change']

    if pd.notna(mamdani_change):
        if mamdani_change >= 0:
            fill_color = pos_colormap(mamdani_change)
        else:
            fill_color = neg_colormap(abs(mamdani_change))

        fill_opacity = 0.7
        line_color = 'black'
        line_weight = 0.5
        dash_array = ''
    else:
        fill_color = 'transparent'
        fill_opacity = 0.4
        line_color = 'red'
        line_weight = 0.5
        dash_array = '5, 5'

    return {
        'fillColor': fill_color,
        'fillOpacity': fill_opacity,
        'color': line_color,
        'weight': line_weight,
        'dashArray': dash_array
    }

# For hover effect
highlight_function = lambda x: {
    'fillColor': '#ab82c5',
    'color': '#7b5aa6',
    'fillOpacity': 0.6,
    'weight': 2,
    'dashArray': ''
}


# ---- Create Folium map ----
if ed_map_gdf_2way.crs and ed_map_gdf_2way.crs.to_epsg() != 4326:
    ed_map_gdf_2way = ed_map_gdf_2way.to_crs(epsg=4326)

center_point = ed_map_gdf_2way.geometry.union_all().centroid
m_change = folium.Map(location=[center_point.y, center_point.x], tiles="Cartodb Positron", zoom_start=11)

# Create formatted percentage columns for tooltips
ed_map_gdf_2way['Mamdani(%)_Primary_fmt'] = ed_map_gdf_2way['Mamdani(%)_Primary'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')
ed_map_gdf_2way['Mamdani(%)_General_fmt'] = ed_map_gdf_2way['Mamdani(%)_General'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')
ed_map_gdf_2way['Mamdani(%)_Change_fmt'] = ed_map_gdf_2way['Mamdani(%)_Change'].apply(lambda x: f'{x:.2f}%' if pd.notna(x) else '')

# Define columns for the tooltip
tooltip_cols_change = [
    'ElectDist',
    'Mamdani(%)_Change_fmt',
    'Mamdani(%)_Primary_fmt',
    'Mamdani(%)_General_fmt',
]

tooltip_aliases_change = [
    'Election District',
    'Mamdani Vote Share Change',
    ' -- Primary Election Share',
    ' -- General Election Share',
]

folium.GeoJson(
    ed_map_gdf_2way.__geo_interface__,
    name='Two-Way Vote Share Change',
    style_function=change_style_function,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols_change,
        aliases=tooltip_aliases_change,
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    )
).add_to(m_change)


# Generate HTML for each colormap
pos_legend_html = pos_colormap._repr_html_()
neg_legend_html = neg_colormap_legend._repr_html_()

# ---- Helper function to filter tick labels ----
def filter_legend_text(html_str, keep_values):
    # Regex to find <text ...>number</text>
    # Matches float or integer numbers inside text tags
    pattern = r'(<text [^>]*>)\s*([\d\.-]+)\s*(</text>)'

    def replacer(m):
        try:
            val = float(m.group(2))
            # Keep if value is close to any in keep_values
            if any(abs(val - k) < 0.1 for k in keep_values):
                return m.group(0)
        except ValueError:
            pass
        return "" # Remove the text tag if not in keep_values

    return re.sub(pattern, replacer, html_str)

# Apply filters
# Keep 20, 40 for positive
pos_legend_html = filter_legend_text(pos_legend_html, [20, 40])
# Keep -20, -40 for negative
neg_legend_html = filter_legend_text(neg_legend_html, [-20, -40])

# Remove negative signs from the negative legend HTML
neg_legend_html = re.sub(r'>-(\d)', r'>\1', neg_legend_html)

# Combine them in a custom HTML string with CSS for positioning
custom_legend_html = f"""
<div style="position: fixed;
            bottom: 20px; right: 6px; width: 466px; height: 277px;
            border:0; z-index:9999; font-size:12px;
            background-color:#fafaf8; opacity:0.8;">
  <div style="padding: 10px;">
    <div style="text-align: center;">
      <h2>Primary&nbsp;&nbsp;→&nbsp;&nbsp;General Election</h2>
      <h2>Two-Way Vote Share Shift in Percentage Points</h2>
      <br>
      <h3>&nbsp;&nbsp;&nbsp;⇦&nbsp;&nbsp;Δ%&nbsp;Toward Cuomo&nbsp;&nbsp;|&nbsp;&nbsp;Δ%&nbsp;Toward Mamdani&nbsp;&nbsp;⇨</h3>
      <div style="display: flex; align-items: center; justify-content: center; gap: 0px;">
        <div>{neg_legend_html}</div>
        <div>|</div>
        <div>{pos_legend_html}</div>
      </div>
    </div>
    <br>
    <div style="line-height: 1.25;">
      <p><b>Note:</b> Two-way vote share for each candidate means that the candidate's vote counts divided by the total votes of both candidates.</p>
      <p><b>Data Source:</b> NYC Board of Elections Unofficial Election Night Results (uncertified and unranked); NYC Department of City Planning</p>
      <p>&copy;&nbsp;<a href="https://r-li.com">Ruoyu Li</a>&nbsp;&nbsp;|&nbsp;&nbsp;Project Repository: <a href="https://github.com/RY-Li/map-nyc-election/">RY-Li/map-nyc-election</a></p>
    </div>
  </div>
</div>
"""

m_change.get_root().html.add_child(folium.Element(custom_legend_html))

# Display the map
m_change

Output hidden; open in https://colab.research.google.com to view.

## Export as HTML

In [221]:
m_change.save('vote_shift.html')
print("Map saved as 'vote_shift.html'")

Map saved as 'vote_shift.html'
